# Imviz profiling

Create three channels of smoothly varying images:

In [8]:
%%time
from time import sleep

import numpy as np
from scipy.ndimage import gaussian_filter
from astropy.nddata import NDData
from astropy.wcs import WCS

from jdaviz import Imviz

shape = (4000, 4000)

wcs = WCS({
    'CTYPE1': 'RA---TAN', 
    'CUNIT1': 'deg', 
    'CDELT1': 0.01,
    'CRPIX1': shape[0] / 2, 
    'CRVAL1': 180,
    'CTYPE2': 'DEC--TAN', 
    'CUNIT2': 'deg', 
    'CDELT2': 0.01,
    'CRPIX2': shape[1] / 2, 
    'CRVAL2': 0
})

random_shape = (6, 6)
image_c = np.random.uniform(size=random_shape)
image_y = np.random.uniform(size=random_shape)
image_m = np.random.uniform(size=random_shape)

colors = ['c', 'y', 'm']

def low_pass_filter(x):
    rfft = np.fft.rfft2(x)
    irfft = np.fft.irfft2(rfft, shape)
    return 2**16 * irfft / irfft.max()

nddata_cym = [
    NDData(low_pass_filter(image), wcs=wcs) 
    for image in [image_c, image_y, image_m]
]

CPU times: user 341 ms, sys: 51.9 ms, total: 393 ms
Wall time: 392 ms


Initialize and show Imviz:

In [10]:
%%time

imviz = Imviz()
label_mouseover = imviz.app.session.application._tools['g-coords-info']
viewer = imviz.default_viewer._obj 
viewer.figure_widget  # SCREENSHOT
imviz.show(height=800)  # EXCLUDE
# imviz.show('sidecar:split-right', height=800)
imviz.app.layout.border = '2px solid rgb(143, 56, 201)'

Application(config='imviz', docs_link='https://jdaviz.readthedocs.io/en/v4.0.0/imviz/index.html', events=['cal…

CPU times: user 637 ms, sys: 110 ms, total: 747 ms
Wall time: 714 ms


You can optionally set the compression to one of these two settings:

In [4]:
imviz.app

Application(config='imviz', docs_link='https://jdaviz.readthedocs.io/en/v4.0.0/imviz/index.html', events=['cal…

In [7]:
viewer = imviz.app.get_viewer_by_id('imviz-0')
viewer.figure_widget

Figure(axes=[Axis(grid_lines='none', label='x', scale=LinearScale(allow_padding=False, max=1.0, min=0.0), side…

In [16]:
# imviz.app.get_viewer('imviz-0')._composite_image.compression = 'png'

# imviz.app.get_viewer('imviz-0')._composite_image.compression = 'none'

Load the images into Imviz:

In [17]:
%%time

with imviz.batch_load():
    for nddata, data_label in zip(nddata_cym, colors):
        imviz.load_data(nddata, data_label=data_label)

compression = imviz.app.get_viewer('imviz-0')._composite_image.compression
print(f"glue-jupyter compression configuration: '{compression}'")

glue-jupyter compression configuration: 'png'
CPU times: user 827 ms, sys: 11.8 ms, total: 838 ms
Wall time: 817 ms


Use WCS to align the images, zoom to fit:

In [18]:
%%time

orientation = imviz.plugins['Orientation']
orientation.align_by = 'WCS'
viewer.zoom_level = 'fit'

CPU times: user 3.09 s, sys: 612 ms, total: 3.7 s
Wall time: 2.54 s


Rotate the image orientation by 180 deg counter-clockwise:

In [19]:
%%time

# rotate the image by 180 deg (CCW):
orientation.rotation_angle = 180
orientation.add_orientation()

CPU times: user 2.04 s, sys: 367 ms, total: 2.41 s
Wall time: 1.74 s


Assign colors to each of the layers:

In [20]:
%%time

plot_options = imviz.plugins["Plot Options"]
for layer, color in zip(plot_options.layer.choices, colors):
    plot_options.layer = layer
    plot_options.image_color_mode = 'Color'
    plot_options.image_color = color
    plot_options.image_opacity = 0.7

CPU times: user 1.45 s, sys: 16.2 ms, total: 1.47 s
Wall time: 1.45 s


Return to the default orientation:

In [21]:
%%time

orientation.orientation = 'Default orientation'

CPU times: user 1.81 s, sys: 292 ms, total: 2.1 s
Wall time: 1.55 s


Blink through each layer (one visible at a time):

In [22]:
%%time

for i in range(len(plot_options.layer.choices)):
    viewer.blink_once()

CPU times: user 781 ms, sys: 4.36 ms, total: 786 ms
Wall time: 777 ms


Make all layers visible again:

In [23]:
%%time

for layer in plot_options.layer.choices:
    plot_options.layer = layer
    plot_options.image_visible = True

CPU times: user 414 ms, sys: 7.44 ms, total: 422 ms
Wall time: 415 ms


Increment through colormap upper limits for each layer:

In [24]:
%%time

for vmax in np.linspace(0, 2**16, 10):
    for layer in plot_options.layer.choices:
        plot_options.layer = layer
        plot_options.stretch_vmin = 0
        plot_options.stretch_vmax = vmax

CPU times: user 5.5 s, sys: 42.4 ms, total: 5.54 s
Wall time: 5.45 s
